#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [2]:
import pandas as pd
import pymongo

In [3]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [6]:
#df_ks.isna().sum()
#df_ks.head()
df_ks = df_ks.drop(['ID', 'usd pledged'], axis = 1)
df_ks.tail()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real
149995,Scutes Headphones - A new way to listen to you...,Product Design,Design,USD,2014-11-14,40000,2014-10-14 20:44:37,3664,canceled,43,US,3664.00
149996,My Drunk @$$ Reviews,Webseries,Film & Video,USD,2012-05-18,1000,2012-04-18 04:42:47,0,failed,0,US,0.00
149997,Freedom Planet - High Speed Platform Game,Video Games,Games,USD,2013-02-15,2000,2013-01-16 00:53:32,25472.7,successful,1105,US,25472.69
149998,Clown vs Mime,Webseries,Film & Video,USD,2011-05-27,1000,2011-04-27 01:22:52,76,failed,3,US,76.00
149999,Striker's Cafe & Bakery,Food Trucks,Food,USD,2016-05-02,20000,2016-04-03 08:01:57,29,failed,3,US,29.00


In [7]:
database.collection.aggregate([
    { "$addFields": {
        "launched": {
            "$toDate": "$launched"
        }
    } }
])

In [9]:
type(database['launched'][0])

pymongo.collection.Collection

### Importer les données

In [10]:
collection.drop()

In [11]:
collection.insert_many(df_ks.to_dict('records'))

In [118]:
#database.collection_names()

## Question 1  

L'idée ici pour récupérer les 5 projets ayant reçu le plus de promesse de dons est de trier dans l'ordre décroissant selon "pledged". On a donc des données dans l'ordre que nous voulons. Il reste alors à afficher les 5 premiers.

In [12]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fce3efeefaf02f4144c8cef'),
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': '2016-05-10 15:53:56',
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd_pledged_real': 9996.0},
 {'_id': ObjectId('5fce3efeefaf02f4144c3bcd'),
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': '2014-10-23 20:06:37',
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd_pledged_real': 9995.0},
 {'_id': ObjectId('5fce3efdefaf02f4144c137d'),
  'name': 'Polco: A Digital Platform for Better Civic Communication',
  'category': 'Web',
  'main_category': 'Technology',
  'currency': 'USD',
  'deadline': '2017-08-11',
  'goal': '35000.0',
  'launche

## Question 2

Pour compter le nombre de projets ayant atteint leur but, on utilise ici `count`.

In [13]:
cur = collection.count({"state":"successful"})
print(cur)

53040


On obtient alors ce résultat. Cependant, il serait préférable de vérifier la cohérence. En effet, s'il y a plus de projets qui ont atteint leur but que de projets dans les données, on a une incohérence.

In [121]:
print(df_ks.shape)

(150000, 12)


On a 53040 < 150000. Notre résultat est donc cohérent.

## Question 3

On cherche à compter le nombre de projets dans chaque catégorie. On utilise alors une aggrégation (`aggregate`). Cette fonction permet d'afficher le nom des catégories ainsi que le nombre de projets dans chaque d'elles.

*Remarque : Il aurait été possible d'utiliser `main_category` à la place de `category`.*

In [14]:
cur = database.kickstarter.aggregate([{"$group" : {"_id" : "$category", "nombre dans categorie" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Camera Equipment', 'nombre dans categorie': 165},
 {'_id': 'Audio', 'nombre dans categorie': 164},
 {'_id': 'Indie Rock', 'nombre dans categorie': 2192},
 {'_id': 'Poetry', 'nombre dans categorie': 532},
 {'_id': 'Flight', 'nombre dans categorie': 158},
 {'_id': 'Farms', 'nombre dans categorie': 482},
 {'_id': 'Art Books', 'nombre dans categorie': 1065},
 {'_id': 'Animals', 'nombre dans categorie': 97},
 {'_id': 'Classical Music', 'nombre dans categorie': 1064},
 {'_id': 'Art', 'nombre dans categorie': 3358},
 {'_id': 'Pet Fashion', 'nombre dans categorie': 51},
 {'_id': 'Woodworking', 'nombre dans categorie': 433},
 {'_id': 'Webcomics', 'nombre dans categorie': 259},
 {'_id': 'Couture', 'nombre dans categorie': 108},
 {'_id': 'Installations', 'nombre dans categorie': 178},
 {'_id': 'Sound', 'nombre dans categorie': 255},
 {'_id': 'Hip-Hop', 'nombre dans categorie': 1555},
 {'_id': 'Horror', 'nombre dans categorie': 525},
 {'_id': 'Kids', 'nombre dans categorie': 109},
 {'_id

## Question 4

Quant à cette question, nous devions compter les projets français réalisés avant 2016. Pour ce faire, l'utilisation de l'opérateur logique `and` nous permet de faire une requête complexe sur notre collection.

In [15]:
cur=collection.find({"$and":[{"country":"FR"}, {"launched":{"$lt":2016}}]})
print(cur.count())

0


Cependant, nous ne trouvons aucun projet français instancié avant 2016. Lorsque l'on indente `$lte`, `$gt` ou `$gte` à la place de `$lt`, nous trouvons aussi 0.

Or, lorsqu'on écrit `$ne` à la place de `$lt`, le résultat affiche 1173. Ceci ne semble pas cohérent.

## Question 5

Il est ici question de lister les projets américains qui ont demandé plus de 200000 dollars. Pour ce faire, il est utile d'utiliser l'opérateur logique `and` qui permet de faire une requête complexe.

In [16]:
cur=collection.find({"$and":[{"country":"US"}, {"usd_pledged_real" : {"$gte" : 200000}}]})
list(cur)

a1e'),
  'name': 'Bears Ears Education Center',
  'category': 'Civic Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2018-01-01',
  'goal': 100000.0,
  'launched': '2017-11-21 21:17:51',
  'pledged': 217243.56,
  'state': 'successful',
  'backers': 3009,
  'country': 'US',
  'usd_pledged_real': 217243.56},
 {'_id': ObjectId('5fce3efeefaf02f4144cda60'),
  'name': 'Boss Monster: The Next Level',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2015-04-28',
  'goal': 25000.0,
  'launched': '2015-04-13 15:03:50',
  'pledged': 335618.85,
  'state': 'successful',
  'backers': 6891,
  'country': 'US',
  'usd_pledged_real': 335618.85},
 {'_id': ObjectId('5fce3efeefaf02f4144cdbab'),
  'name': 'Martians: A Story of Civilization',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-08-01',
  'goal': 30000.0,
  'launched': '2016-07-11 17:00:41',
  'pledged': 220932.0,
  'state'

## Question 6 

Dans cette question, il est demandé de compter le nombre de projets ayant "Sport" dans leur nom.

In [17]:
#df_ks['name']=df_ks['name'].to_list()
cur = collection.find({"name":{"$all":["Sport"]}}).count()
print(cur)

0


Ici, on trouve qu'aucun projet ne contient "Sport" dans son nom. Cela est peut-être dû au fait que cette requête cherche les projets ayant pour nom "Sport". Pour pallier à ce problème, j'ai pensé qu'il fallait peut être se rapprocher au cas vu dans le cours, et découper les noms en liste. Cependant, cela ne marche toujours pas.